In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error, auc

from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar

import numpy as np
import pandas as pd

### **Step 01: Data Wrangling

Imported data sets here are about the third repurchase rate (from second to third purchase) of several cohorts of the customers in four different types of Order Dynamics and three different brands. The data set has not been updated since November 2021.

Cohorts are based on the first purchase month of each customer. All the customers who made the first purchase in certain brand in the same month are grouped in one cohort.

Order Dynamics: composed of Onetime Order and Subsription
- OO: Onetime Order -> Onetime Order
- OS: Onetime Order -> Subscription
- SS: Subscription -> Subscription
- SO: Subscirption -> Subscription

In [2]:
# set base dir
# import .csv file 
# loaded csv file is about repurchase rate in the Subscription to Subscription Order Dynamics of all cohorts
# by loading different files, we can conduct the prediction of the other types of Order Dynamics
df_RR = pd.read_csv('SubscriptionToSubcription_RepurchaseSecondToThird.csv', index_col=0)

In [3]:
df_RR.head()

,avgDaysPast,N2pop,N3pop,hist_RR,d0,d1,d2,d3,d4,d5,...,d711,d712,d713,d714,d715,d716,d717,d718,d719,d720
cohort,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1179.73,232,204,87.93%,0.00%,0.43%,0.43%,0.43%,0.43%,0.43%,...,90.45%,90.45%,90.45%,90.45%,90.45%,90.45%,90.45%,90.45%,90.45%,90.45%
2018-02-01,1084.37,3246,2792,86.01%,0.18%,0.18%,0.22%,0.22%,0.25%,0.25%,...,88.65%,88.65%,88.65%,88.65%,88.65%,88.64%,88.64%,88.64%,88.64%,88.63%
2018-03-01,1080.56,5135,4405,85.78%,0.18%,0.18%,0.18%,0.18%,0.18%,0.21%,...,88.16%,88.16%,88.16%,88.16%,88.18%,88.19%,88.21%,88.21%,88.21%,88.23%
2018-04-01,1059.69,6039,5117,84.73%,0.26%,0.28%,0.30%,0.33%,0.36%,0.40%,...,87.06%,87.07%,87.06%,87.13%,87.12%,87.11%,87.11%,87.11%,87.11%,87.13%
2018-05-01,1040.14,6509,5596,85.97%,0.22%,0.25%,0.28%,0.28%,0.29%,0.34%,...,88.53%,88.53%,88.53%,88.53%,88.52%,88.52%,88.52%,88.52%,88.51%,88.50%


In [4]:
# preprocess DataFrame for use
avgDaysPast = df_RR.avgDaysPast.to_dict()
df_RR = df_RR.loc[:, 'd0':'d420'].transpose()
df_RR.replace('[%]', r'', regex=True, inplace=True)
df_RR = df_RR.apply(pd.to_numeric)
df_RR.index.name = 'days cnt'

# remove cohorts with abnormal pattern in Repurchase Rate change
# df_RR.drop(columns='2021-06-01', axis=1, inplace=True)
df_RR.drop(columns='2021-07-01', axis=1, inplace=True)
df_RR.drop(columns='2021-08-01', axis=1, inplace=True)
df_RR.drop(columns='2021-09-01', axis=1, inplace=True)
df_RR.drop(columns='2021-10-01', axis=1, inplace=True)
df_RR.reset_index(drop=True, inplace=True)

In [5]:
#datetime.strptime("2021-01-01", '%Y-%m-%d')+relativedelta(month = 2)-relativedelta(days=1)
df_RR.head()

cohort,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,...,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01
0,0.00,0.18,0.18,0.26,0.22,0.21,0.17,0.14,0.23,0.25,...,0.23,0.13,0.11,0.26,0.26,0.30,0.17,0.12,0.4,0.51
1,0.43,0.18,0.18,0.28,0.25,0.24,0.23,0.16,0.23,0.29,...,0.25,0.21,0.16,0.37,0.26,0.30,0.17,0.19,0.4,0.51
2,0.43,0.22,0.18,0.30,0.28,0.24,0.27,0.17,0.27,0.35,...,0.25,0.21,0.20,0.42,0.33,0.37,0.18,0.31,0.4,0.61
3,0.43,0.22,0.18,0.33,0.28,0.27,0.27,0.17,0.27,0.37,...,0.27,0.26,0.22,0.42,0.33,0.45,0.18,0.44,0.4,0.61
4,0.43,0.25,0.18,0.36,0.29,0.30,0.27,0.17,0.29,0.37,...,0.27,0.30,0.24,0.47,0.33,0.52,0.21,0.56,0.4,0.61


In [6]:
# replace `NA` null-values with the current date
cohort_all = list(df_RR.columns)
date_cohort = []
now = datetime.now()

# Calculate `days_past` value 
for i, cohort in enumerate(cohort_all):
    #print(cohort)
    #define "avgdp30mult" value as the smallest multiples of 30 closest to "avgDaysPast" value
    # (since it is the period with the most stable Repurchase Rate)
    avgdp30mult = int((avgDaysPast[cohort]//30)*30)

    # set `NA` null-values for the df_RR DataFrame
    if avgdp30mult < 420:
        df_RR.iloc[avgdp30mult:, i] = np.nan

df_RR_prefill = df_RR.copy(deep=True)

### **Step 02: `Curve-Fitting` for Base Cohorts**

In [7]:
# Fitting Function - rational function
def func_rational(x, a, b, c, d):
    return np.polyval([a,b], x) / np.polyval([c,d], x)

# take arbitrary values as a initial settings of curve-fitting parameters
initialGuess_rational = [0.1, 0.1, 0.1, 0.1]

In [8]:
# store the fitted base cohorts into a new data frame "df_RR_basefit"
df_RR_basefit = pd.DataFrame()

In [9]:
# Custom curve-fitting function for all base cohorts with over 720 days past 
# (regard "720 days = about 2 years" as desertion period
def fit_basecohort():
    cohort_base = list(df_RR.loc[:, (df_RR.isnull().sum() == 0)].columns)
    list_days = list(range(0, 421))

    # empty df_RR_basefit 
    df_RR_basefit.drop(index=df_RR_basefit.index, inplace=True)
    
    # Set df_RR_basefit
    df_RR_basefit['cohort'] = cohort_base
    df_RR_basefit.set_index('cohort', inplace=True)

    for cohort in cohort_base:
        # set up X,Y Data
        xBase = list_days
        yBase = df_RR.loc[:, cohort]
        
        # Perform Curve-fitting 
        popt_base, pcov_base = curve_fit(func_rational, xBase, yBase, initialGuess_rational)
        # Based on this point, we can adjust the curve-fitting parameters

        # Update DataFrame with fitted parameters
        df_RR_basefit.loc[cohort, 'fitted_a'] = popt_base[0]
        df_RR_basefit.loc[cohort, 'fitted_b'] = popt_base[1]
        df_RR_basefit.loc[cohort, 'fitted_c'] = popt_base[2]
        df_RR_basefit.loc[cohort, 'fitted_d'] = popt_base[3]

### **Step 03: `Fill` the null-values in each cohort with curve-fitting method**

In [10]:
# Select all cohorts that needs to be filled 
cohort_fill = list(df_RR.loc[:, (df_RR.isnull().sum() > 0)].columns)

# Create DataFrame with trimmed Repurchase Rate data 
df_RR_shortTerm = pd.DataFrame()

# Create DataFrame for Short-term Repurchase Rate Comparison
df_RR_compare = pd.DataFrame({'fit_cohort':cohort_fill, 'UB_cohort':'', 'UB_MSE':10000, 'LB_cohort':'', 'LB_MSE':10000, 'UB_SUB_cohort':'', 'UB_SUB_MSE':10000, 'LB_SUB_cohort':'', 'LB_SUB_MSE':10000})
df_RR_compare.set_index('fit_cohort', inplace=True)

# Optimize the variables with weight tuning 
tuning_range = 0.01 
tuning_external_range = 4
optimal_param = [0.0, 0.0, 0.0, 0.0]

In [11]:
# Loop & Compare by MSE and AUC (Mean-Squared-Error and Area-Under-Curve)
def compare_shortTerm(fill_cohort, compare_cohort, fit_days, AUC_my):

    dFit = np.linspace(0, fit_days, fit_days+1)

    # Update df_RR_compare based on AUC and lowest MSE
    for compare in compare_cohort:
        # Calculate AUC and MSE of the comparing cohorts       
        AUC_compare = auc(dFit, df_RR.loc[:days_tofit, compare]) 
        MSE = mean_squared_error(df_RR.loc[:fit_days, fill_cohort], df_RR_shortTerm[compare])
        #print("Cohort :", compare, " AUC :", AUC_compare, " MSE :",MSE)

        # Check the cohorts near the upper-bound
        if AUC_compare > AUC_my:
            if MSE < df_RR_compare.loc[fill_cohort, 'UB_MSE']:
                # Push current UB to UB_SUB
                df_RR_compare.loc[fill_cohort, 'UB_SUB_MSE'] = df_RR_compare.loc[fill_cohort, 'UB_MSE']
                df_RR_compare.loc[fill_cohort, 'UB_SUB_cohort'] = df_RR_compare.loc[fill_cohort, 'UB_cohort']
                # Update UB
                df_RR_compare.loc[fill_cohort, 'UB_MSE'] = MSE
                df_RR_compare.loc[fill_cohort, 'UB_cohort'] = compare
            else:
                # For the case when the new MSE is larger than UB_MSE -> Push new MSE in SUB_MSE if new MSE is less than SUB_MSE
                if MSE < df_RR_compare.loc[fill_cohort, 'UB_SUB_MSE']:
                    df_RR_compare.loc[fill_cohort, 'UB_SUB_MSE'] = MSE
                    df_RR_compare.loc[fill_cohort, 'UB_SUB_cohort'] = compare
                else : 
                    pass
        # Check the cohorts near lower-bound
        elif AUC_compare < AUC_my:
            if MSE < df_RR_compare.loc[fill_cohort, 'LB_MSE']:
                # Push current LB to LB_SUB
                df_RR_compare.loc[fill_cohort, 'LB_SUB_MSE'] = df_RR_compare.loc[fill_cohort, 'LB_MSE']
                df_RR_compare.loc[fill_cohort, 'LB_SUB_cohort'] = df_RR_compare.loc[fill_cohort, 'LB_cohort']
                # Update LB
                df_RR_compare.loc[fill_cohort, 'LB_MSE'] = MSE
                df_RR_compare.loc[fill_cohort, 'LB_cohort'] = compare
            else:
                # For the case when the new MSE is greater than LB_MSE -> Push new MSE in SUB_MSE if new MSE is less than SUB_MSE
                if MSE < df_RR_compare.loc[fill_cohort, 'LB_SUB_MSE']:
                    df_RR_compare.loc[fill_cohort, 'LB_SUB_MSE'] = MSE
                    df_RR_compare.loc[fill_cohort, 'LB_SUB_cohort'] = compare
                else : 
                    pass
        

In [12]:
# Define customized MSE calculation function based on the given weighted values
def get_weighted_MSE(base_cohort, fit_day, weight_1, weight_2):
    cohort_UB, cohort_LB = df_RR_compare.loc[base_cohort, 'UB_cohort'], df_RR_compare.loc[base_cohort, 'LB_cohort']

    # Highest Cohort 
    if cohort_UB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'LB_SUB_cohort']
        weighted_y = (weight_1 * df_RR_shortTerm[cohort_LB]) + (weight_2 * df_RR_shortTerm[cohort_SUB])
    # Lowest Cohort
    elif cohort_LB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'UB_SUB_cohort']
        weighted_y = (weight_1 * df_RR_shortTerm[cohort_UB]) + (weight_2 * df_RR_shortTerm[cohort_SUB])
    # Normal
    else:
        weighted_y = (weight_1 * df_RR_shortTerm[cohort_UB]) + (weight_2 * df_RR_shortTerm[cohort_LB])
    
    return mean_squared_error(df_RR.loc[:fit_day, base_cohort], weighted_y)

# Define customized weighted parameter function based on the given weights
def get_weighted_param(base_cohort, weight_1, weight_2):
    cohort_UB, cohort_LB = df_RR_compare.loc[base_cohort, 'UB_cohort'], df_RR_compare.loc[base_cohort, 'LB_cohort']
    
    # Highest Cohort 
    if cohort_UB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'LB_SUB_cohort']
        weighted_param = (weight_1 * df_RR_basefit.loc[cohort_LB, :]) + (weight_2 * df_RR_basefit.loc[cohort_SUB, :])
    # Lowest Cohort
    elif cohort_LB == '':
        cohort_SUB = df_RR_compare.loc[base_cohort, 'UB_SUB_cohort']
        weighted_param = (weight_1 * df_RR_basefit.loc[cohort_SUB, :]) + (weight_2 * df_RR_basefit.loc[cohort_UB, :])
    # Normal
    else:
        weighted_param = (weight_1 * df_RR_basefit.loc[cohort_UB, :]) + (weight_2 * df_RR_basefit.loc[cohort_LB, :])
    
    return weighted_param

In [13]:
# Loop & find the optimal parameter for Repurchase Rate prediction
def optimize_parameter(base_cohort, fit_day):
    cohort_UB, cohort_LB = df_RR_compare.loc[base_cohort, 'UB_cohort'], df_RR_compare.loc[base_cohort, 'LB_cohort']
    optimal_MSE, optimal_weight = 10000, [0.0, 0.0]

    # Case: Normal 
    if (cohort_UB != '') & (cohort_LB != ''):
        for i in range(int(1/tuning_range+1)):
            test_weight = [(i*tuning_range), (1-(i*tuning_range))]
            test_MSE = get_weighted_MSE(base_cohort, fit_day, *test_weight)
            
            # Update the optimal values
            if test_MSE < optimal_MSE:
                optimal_MSE = test_MSE
                optimal_weight = test_weight[:]    
    
    # Case: Highest or Lowest 
    else:
        for i in range(int(1/tuning_range+1)*(tuning_external_range-1)):
            # Case: Highest
            if cohort_UB == '':
                test_weight = [(tuning_external_range-i*tuning_range), (1-(tuning_external_range-i*tuning_range))]
            # Case: Lowest
            elif cohort_LB == '':
                test_weight = [(1-(tuning_external_range-i*tuning_range)), (tuning_external_range-i*tuning_range)]

            test_MSE = get_weighted_MSE(base_cohort, fit_day, *test_weight)
            
            # Update the optimal values
            if test_MSE < optimal_MSE:
                optimal_MSE = test_MSE
                optimal_weight = test_weight[:] 

    #print('optimal_weight:\t', optimal_weight, '\t', 'optimal_MSE:\t', optimal_MSE)
    return get_weighted_param(base_cohort, *optimal_weight)

In [14]:
# update 'null' value in cohorts with fitted parameters 
def update_nullValue(fill_cohort, fit_day, optimalparam):
    
    # Pass optimal paratmeter to rational_function
    xFit = np.linspace(0, 420, 421)
    yPred = func_rational(xFit, *optimalparam)

    # update `null` value in cohorts 
    df_RR.loc[(fit_day+1):, fill_cohort] = yPred[(fit_day+1):]

    chk = (df_RR.loc[fit_day, fill_cohort] - yPred[(fit_day+1)])

    if (chk >= 0) & (abs(chk) > 0.1):
        df_RR.loc[(fit_day+1):, fill_cohort] = yPred[(fit_day+1):] + abs(chk)
    
    elif (chk < 0) & (abs(chk) > 0.1):
        df_RR.loc[(fit_day+1):, fill_cohort] = yPred[(fit_day+1):] - abs(chk)

In [16]:
# Loop & Fill all the null values in the cohorts
for fill in cohort_fill:
    # reset the base cohort before every update process
    fit_basecohort()

    # Set up the cohort & days range for the computation
    cohort_compare = df_RR_basefit.index.values
    days_tofit = (420 - df_RR[fill].isnull().sum())
    dFit = np.linspace(0, days_tofit, days_tofit+1)

    # Update df_RR_shortTerm 
    df_RR_shortTerm = df_RR.loc[:days_tofit, cohort_compare]
    AUC_my = auc(dFit, df_RR.loc[:days_tofit, fill]) 
    print(AUC_my)
    compare_shortTerm(fill, cohort_compare, days_tofit, AUC_my)
    
    # Optimize the weighted values on the parameters
    optimal_param = optimize_parameter(fill, days_tofit)

    # update 'null' value in cohorts with fitted parameters 
    update_nullValue(fill, days_tofit, optimal_param)

21016.56
19187.14
17280.885
15424.0
15457.115
12967.68
11621.585000000001
12469.56
11678.484999999999
9353.44
6310.87
4065.4149999999995
2526.4000000000005
1201.305
277.09499999999997


In [17]:
df_RR_compare.head()

,UB_cohort,UB_MSE,LB_cohort,LB_MSE,UB_SUB_cohort,UB_SUB_MSE,LB_SUB_cohort,LB_SUB_MSE
fit_cohort,,,,,,,,
2020-04-01,2020-02-01,35.644254,,10000.000000,2020-01-01,36.953708,,10000.000000
2020-05-01,2020-02-01,29.070141,2020-04-01,1.489384,2020-01-01,29.281520,,10000.000000
2020-06-01,2020-02-01,18.556712,2020-05-01,2.043124,2020-01-01,19.024860,2020-04-01,4.899302
2020-07-01,2020-01-01,12.325674,2020-06-01,1.987239,2020-02-01,12.535284,2020-05-01,6.758816
2020-08-01,2020-01-01,12.386717,2020-07-01,0.279671,2020-02-01,13.320888,2020-06-01,2.736222


In [18]:
fig_raw = px.line(data_frame=df_RR_prefill)
fig_raw.show()
#fig_raw.write_html("SubscriptionToSubcription_RepurchaseSecondToThird_Raw.html")

In [19]:
fig_pred = px.line(data_frame=df_RR)
fig_pred.show()
#fig_pred.write_html("SubscriptionToSubcription_RepurchaseSecondToThird_Predicted.html")

In [133]:
#df_RR.iloc[420].to_csv("SubscriptionToSubcription_RepurchaseSecondToThird_Predicted.csv", encoding= "cp949")

In [20]:
px.line(data_frame=df_RR)

In [26]:
# Save predicted results as .csv format
today = datetime.today()
today = today.strftime('%Y%m%d')
df_RR_save = df_RR.loc[510: :].transpose()
#df_RR_save.to_csv('LTV_Pred_Result'+today+'.csv')

# df_RR.loc[720, :].to_csv('LTV_Pred_Result'+today+'.csv')